# Practise Case 01 - Covid-19 data analysis with pandas

First we want to practise loading online csv files.
The file is provided by [ourworldindata.org](https://covid.ourworldindata.org/data/owid-covid-data.csv) and updated daily. The following analysis could be run as a daily job. 

**CASE GOAL:**
Find the 10 countries with highest case numbers per million inhabitants. Give the output as json.

## 0. Import necessary packages

In [1]:
import pandas as pd
from functools import reduce
from datetime import datetime, timedelta

## 1. Load data

As the data may become increasingly larger, I consider to load it in chunks instead of all together. I want to compare the time it takes to load it this way vs. in-built pandas functionality.


In [2]:
url = "https://covid.ourworldindata.org/data/owid-covid-data.csv"

In [3]:
# read the full file 
raw = pd.read_csv(url)

In [4]:
time pd.read_csv(url)

CPU times: user 623 ms, sys: 154 ms, total: 777 ms
Wall time: 3.37 s


,iso_code,continent,location,date,total_cases,new_cases,new_cases_smoothed,total_deaths,new_deaths,new_deaths_smoothed,...,extreme_poverty,cardiovasc_death_rate,diabetes_prevalence,female_smokers,male_smokers,handwashing_facilities,hospital_beds_per_thousand,life_expectancy,human_development_index,excess_mortality
0,AFG,Asia,Afghanistan,2020-02-24,1.0,1.0,NaN,NaN,NaN,NaN,...,NaN,597.029,9.59,NaN,NaN,37.746,0.5,64.83,0.511,NaN
1,AFG,Asia,Afghanistan,2020-02-25,1.0,0.0,NaN,NaN,NaN,NaN,...,NaN,597.029,9.59,NaN,NaN,37.746,0.5,64.83,0.511,NaN
2,AFG,Asia,Afghanistan,2020-02-26,1.0,0.0,NaN,NaN,NaN,NaN,...,NaN,597.029,9.59,NaN,NaN,37.746,0.5,64.83,0.511,NaN
3,AFG,Asia,Afghanistan,2020-02-27,1.0,0.0,NaN,NaN,NaN,NaN,...,NaN,597.029,9.59,NaN,NaN,37.746,0.5,64.83,0.511,NaN
4,AFG,Asia,Afghanistan,2020-02-28,1.0,0.0,NaN,NaN,NaN,NaN,...,NaN,597.029,9.59,NaN,NaN,37.746,0.5,64.83,0.511,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
101542,ZWE,Africa,Zimbabwe,2021-07-06,57963.0,1949.0,1347.143,1939.0,28.0,25.429,...,21.4,307.846,1.82,1.6,30.7,36.791,1.7,61.49,0.571,NaN
101543,ZWE,Africa,Zimbabwe,2021-07-07,60227.0,2264.0,1480.429,1973.0,34.0,26.286,...,21.4,307.846,1.82,1.6,30.7,36.791,1.7,61.49,0.571,NaN
101544,ZWE,Africa,Zimbabwe,2021-07-08,62383.0,2156.0,1594.571,2029.0,56.0,31.571,...,21.4,307.846,1.82,1.6,30.7,36.791,1.7,61.49,0.571,NaN
101545,ZWE,Africa,Zimbabwe,2021-07-09,65066.0,2683.0,1771.857,2084.0,55.0,34.714,...,21.4,307.846,1.82,1.6,30.7,36.791,1.7,61.49,0.571,NaN


## 2. Transform & Load in chunks

In [5]:
today = datetime.now().date()
yesterday = today + timedelta(days=-1)

In [6]:
# read the file in chunks and do some basic transformation

cols = ['iso_code', 'continent', 'location', 'date', 'total_cases', 
        'new_cases', 'new_cases_per_million', 'population']

def read_chunks(url, cols):
    raw_TL = None
    for chunk in pd.read_csv(url, usecols=cols, chunksize=1000):
        chunk['date'] = pd.to_datetime(chunk['date'])
        chunk_result = chunk[chunk['date']== pd.to_datetime(yesterday)]
        chunk_result['assert_new_cases_per_million'] = chunk_result['new_cases'] / (chunk_result['population']/1e6)
        if raw_TL is None:
            raw_TL = chunk_result
        else:
            raw_TL = raw_TL.append(chunk_result)
    return raw_TL

raw_TL = read_chunks(url, cols)

  iso_code continent     location        date  total_cases  new_cases  \
0      AFG      Asia  Afghanistan  2020-02-24          1.0        1.0   
1      AFG      Asia  Afghanistan  2020-02-25          1.0        0.0   

   new_cases_per_million  population  
0                  0.026  38928341.0  
1                  0.000  38928341.0  
2021-07-10 00:00:00
True
    iso_code continent     location       date  total_cases  new_cases  \
502      AFG      Asia  Afghanistan 2021-07-10     131586.0        0.0   

     new_cases_per_million  population  assert_new_cases_per_million  
502                    0.0  38928341.0                           0.0  
      iso_code continent location        date  total_cases  new_cases  \
1000  OWID_AFR       NaN   Africa  2021-06-24    5314808.0    33699.0   
1001  OWID_AFR       NaN   Africa  2021-06-25    5350331.0    35523.0   

      new_cases_per_million    population  
1000                 25.137  1.340598e+09  
1001                 26.498  1.340598e+

/var/folders/6c/08v1kd5x5yjgw73zcdxqckzh0000gn/T/ipykernel_11209/4132050757.py:12: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior.  In a future version these will be considered non-comparable.Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.
  print(yesterday == chunk['date'].max())
/Users/lisa-mariekrause/Documents/01_Karriere/02_Bewerbungsunterlagen/Bewerbungen/Quantum_Black/Case_Prep/QB_case/lib/python3.9/site-packages/pandas/core/frame.py:3607: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._set_item(key, value)
/var/folders/6c/08v1kd5x5yjgw73zcdxqckzh0000gn/T/ipykernel_11209/4132050757.py:12: FutureWarning: Comparison of Timestamp with datetime.date is 

      iso_code      continent location       date  total_cases  new_cases  \
11460      BTN           Asia   Bhutan 2021-07-10       2266.0        8.0   
11947      BOL  South America  Bolivia 2021-07-10     453595.0     2371.0   

       new_cases_per_million  population  assert_new_cases_per_million  
11460                 10.368    771612.0                     10.367905  
11947                203.118  11673029.0                    203.117803  
      iso_code continent                location        date  total_cases  \
12000      BIH    Europe  Bosnia and Herzegovina  2020-04-25       1486.0   
12001      BIH    Europe  Bosnia and Herzegovina  2020-04-26       1516.0   

       new_cases  new_cases_per_million  population  
12000       65.0                 19.812   3280815.0  
12001       30.0                  9.144   3280815.0  
2021-07-10 00:00:00
True
      iso_code continent                location       date  total_cases  \
12441      BIH    Europe  Bosnia and Herzegovina 2021-

/var/folders/6c/08v1kd5x5yjgw73zcdxqckzh0000gn/T/ipykernel_11209/4132050757.py:12: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior.  In a future version these will be considered non-comparable.Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.
  print(yesterday == chunk['date'].max())
/Users/lisa-mariekrause/Documents/01_Karriere/02_Bewerbungsunterlagen/Bewerbungen/Quantum_Black/Case_Prep/QB_case/lib/python3.9/site-packages/pandas/core/frame.py:3607: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._set_item(key, value)
/var/folders/6c/08v1kd5x5yjgw73zcdxqckzh0000gn/T/ipykernel_11209/4132050757.py:12: FutureWarning: Comparison of Timestamp with datetime.date is 


True
      iso_code      continent    location       date  total_cases  new_cases  \
21029      COG         Africa       Congo 2021-07-10      12790.0        0.0   
21557      CRI  North America  Costa Rica 2021-07-10     380482.0        0.0   

       new_cases_per_million  population  assert_new_cases_per_million  
21029                    0.0   5518092.0                           0.0  
21557                    0.0   5094114.0                           0.0  
      iso_code continent       location        date  total_cases  new_cases  \
22000      CIV    Africa  Cote d'Ivoire  2021-05-27      47146.0       61.0   
22001      CIV    Africa  Cote d'Ivoire  2021-05-28      47195.0       49.0   

       new_cases_per_million  population  
22000                  2.313  26378275.0  
22001                  1.858  26378275.0  
2021-07-10 00:00:00
True
      iso_code continent       location       date  total_cases  new_cases  \
22044      CIV    Africa  Cote d'Ivoire 2021-07-10      48776.0 

...

/var/folders/6c/08v1kd5x5yjgw73zcdxqckzh0000gn/T/ipykernel_11209/4132050757.py:12: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior.  In a future version these will be considered non-comparable.Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.
  print(yesterday == chunk['date'].max())
/Users/lisa-mariekrause/Documents/01_Karriere/02_Bewerbungsunterlagen/Bewerbungen/Quantum_Black/Case_Prep/QB_case/lib/python3.9/site-packages/pandas/core/frame.py:3607: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._set_item(key, value)
/var/folders/6c/08v1kd5x5yjgw73zcdxqckzh0000gn/T/ipykernel_11209/4132050757.py:12: FutureWarning: Comparison of Timestamp with datetime.date is 

      iso_code continent    location        date  total_cases  new_cases  \
81000      SYC    Africa  Seychelles  2021-04-21       5016.0        4.0   
81001      SYC    Africa  Seychelles  2021-04-22       5170.0      154.0   

       new_cases_per_million  population  
81000                 40.675     98340.0  
81001               1565.996     98340.0  
2021-07-10 00:00:00
True
      iso_code continent      location       date  total_cases  new_cases  \
81080      SYC    Africa    Seychelles 2021-07-10      16679.0        0.0   
81547      SLE    Africa  Sierra Leone 2021-07-10       6003.0       40.0   

       new_cases_per_million  population  assert_new_cases_per_million  
81080                  0.000     98340.0                      0.000000  
81547                  5.014   7976985.0                      5.014426  
      iso_code continent   location        date  total_cases  new_cases  \
82000      SGP      Asia  Singapore  2021-04-19      60851.0       20.0   
82001      SGP  

/var/folders/6c/08v1kd5x5yjgw73zcdxqckzh0000gn/T/ipykernel_11209/4132050757.py:12: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior.  In a future version these will be considered non-comparable.Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.
  print(yesterday == chunk['date'].max())
/Users/lisa-mariekrause/Documents/01_Karriere/02_Bewerbungsunterlagen/Bewerbungen/Quantum_Black/Case_Prep/QB_case/lib/python3.9/site-packages/pandas/core/frame.py:3607: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._set_item(key, value)
/var/folders/6c/08v1kd5x5yjgw73zcdxqckzh0000gn/T/ipykernel_11209/4132050757.py:12: FutureWarning: Comparison of Timestamp with datetime.date is 

      iso_code continent location        date  total_cases  new_cases  \
93000      TUN    Africa  Tunisia  2020-03-06          1.0        0.0   
93001      TUN    Africa  Tunisia  2020-03-07          1.0        0.0   

       new_cases_per_million  population  
93000                    0.0  11818618.0  
93001                    0.0  11818618.0  
2021-07-10 00:00:00
True
      iso_code continent location       date  total_cases  new_cases  \
93491      TUN    Africa  Tunisia 2021-07-10     491021.0     9286.0   
93978      TUR      Asia   Turkey 2021-07-10    5465094.0        0.0   

       new_cases_per_million  population  assert_new_cases_per_million  
93491                785.709  11818618.0                    785.709463  
93978                  0.000  84339067.0                      0.000000  
      iso_code      continent                  location        date  \
94000      TCA  North America  Turks and Caicos Islands  2021-01-30   
94001      TCA  North America  Turks and Caicos 

/var/folders/6c/08v1kd5x5yjgw73zcdxqckzh0000gn/T/ipykernel_11209/4132050757.py:12: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior.  In a future version these will be considered non-comparable.Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.
  print(yesterday == chunk['date'].max())
/Users/lisa-mariekrause/Documents/01_Karriere/02_Bewerbungsunterlagen/Bewerbungen/Quantum_Black/Case_Prep/QB_case/lib/python3.9/site-packages/pandas/core/frame.py:3607: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._set_item(key, value)
/var/folders/6c/08v1kd5x5yjgw73zcdxqckzh0000gn/T/ipykernel_11209/4132050757.py:12: FutureWarning: Comparison of Timestamp with datetime.date is 

In [7]:
time pd.read_csv(url, chunksize=1000)

CPU times: user 153 ms, sys: 105 ms, total: 258 ms
Wall time: 2.97 s


In [8]:
time read_chunks(url, cols)

  iso_code continent     location        date  total_cases  new_cases  \
0      AFG      Asia  Afghanistan  2020-02-24          1.0        1.0   
1      AFG      Asia  Afghanistan  2020-02-25          1.0        0.0   

   new_cases_per_million  population  
0                  0.026  38928341.0  
1                  0.000  38928341.0  
2021-07-10 00:00:00
True
    iso_code continent     location       date  total_cases  new_cases  \
502      AFG      Asia  Afghanistan 2021-07-10     131586.0        0.0   

     new_cases_per_million  population  assert_new_cases_per_million  
502                    0.0  38928341.0                           0.0  
      iso_code continent location        date  total_cases  new_cases  \
1000  OWID_AFR       NaN   Africa  2021-06-24    5314808.0    33699.0   
1001  OWID_AFR       NaN   Africa  2021-06-25    5350331.0    35523.0   

      new_cases_per_million    population  
1000                 25.137  1.340598e+09  
1001                 26.498  1.340598e+

/var/folders/6c/08v1kd5x5yjgw73zcdxqckzh0000gn/T/ipykernel_11209/4132050757.py:12: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior.  In a future version these will be considered non-comparable.Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.
  print(yesterday == chunk['date'].max())
/Users/lisa-mariekrause/Documents/01_Karriere/02_Bewerbungsunterlagen/Bewerbungen/Quantum_Black/Case_Prep/QB_case/lib/python3.9/site-packages/pandas/core/frame.py:3607: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._set_item(key, value)
/var/folders/6c/08v1kd5x5yjgw73zcdxqckzh0000gn/T/ipykernel_11209/4132050757.py:12: FutureWarning: Comparison of Timestamp with datetime.date is 


      iso_code continent                location        date  total_cases  \
12000      BIH    Europe  Bosnia and Herzegovina  2020-04-25       1486.0   
12001      BIH    Europe  Bosnia and Herzegovina  2020-04-26       1516.0   

       new_cases  new_cases_per_million  population  
12000       65.0                 19.812   3280815.0  
12001       30.0                  9.144   3280815.0  
2021-07-10 00:00:00
True
      iso_code continent                location       date  total_cases  \
12441      BIH    Europe  Bosnia and Herzegovina 2021-07-10     205145.0   
12909      BWA    Africa                Botswana 2021-07-10      75388.0   

       new_cases  new_cases_per_million  population  \
12441       98.0                 29.871   3280815.0   
12909        0.0                  0.000   2351625.0   

       assert_new_cases_per_million  
12441                     29.870627  
12909                      0.000000  
      iso_code      continent location        date  total_cases  new_ca

/var/folders/6c/08v1kd5x5yjgw73zcdxqckzh0000gn/T/ipykernel_11209/4132050757.py:12: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior.  In a future version these will be considered non-comparable.Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.
  print(yesterday == chunk['date'].max())
/Users/lisa-mariekrause/Documents/01_Karriere/02_Bewerbungsunterlagen/Bewerbungen/Quantum_Black/Case_Prep/QB_case/lib/python3.9/site-packages/pandas/core/frame.py:3607: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._set_item(key, value)
/var/folders/6c/08v1kd5x5yjgw73zcdxqckzh0000gn/T/ipykernel_11209/4132050757.py:12: FutureWarning: Comparison of Timestamp with datetime.date is 


      iso_code continent       location        date  total_cases  new_cases  \
22000      CIV    Africa  Cote d'Ivoire  2021-05-27      47146.0       61.0   
22001      CIV    Africa  Cote d'Ivoire  2021-05-28      47195.0       49.0   

       new_cases_per_million  population  
22000                  2.313  26378275.0  
22001                  1.858  26378275.0  
2021-07-10 00:00:00
True
      iso_code continent       location       date  total_cases  new_cases  \
22044      CIV    Africa  Cote d'Ivoire 2021-07-10      48776.0        0.0   
22546      HRV    Europe        Croatia 2021-07-10     360768.0       88.0   

       new_cases_per_million  population  assert_new_cases_per_million  
22044                  0.000  26378275.0                      0.000000  
22546                 21.436   4105268.0                     21.435872  
      iso_code      continent location        date  total_cases  new_cases  \
23000      CUB  North America     Cuba  2021-06-08     151259.0     1156.0 

...

/Users/lisa-mariekrause/Documents/01_Karriere/02_Bewerbungsunterlagen/Bewerbungen/Quantum_Black/Case_Prep/QB_case/lib/python3.9/site-packages/pandas/core/frame.py:3607: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._set_item(key, value)
/var/folders/6c/08v1kd5x5yjgw73zcdxqckzh0000gn/T/ipykernel_11209/4132050757.py:12: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior.  In a future version these will be considered non-comparable.Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.
  print(yesterday == chunk['date'].max())
/Users/lisa-mariekrause/Documents/01_Karriere/02_Bewerbungsunterlagen/Bewerbungen/Quantum_Black/Case_Prep/QB_case/lib/python3.9/site-packages/pa

      iso_code continent     location       date  total_cases  new_cases  \
88487      SWE    Europe       Sweden 2021-07-10    1092540.0        0.0   
88989      CHE    Europe  Switzerland 2021-07-10     704943.0        0.0   

       new_cases_per_million  population  assert_new_cases_per_million  
88487                    0.0  10099270.0                           0.0  
88989                    0.0   8654618.0                           0.0  
      iso_code continent location        date  total_cases  new_cases  \
89000      SYR      Asia    Syria  2020-04-01         10.0        0.0   
89001      SYR      Asia    Syria  2020-04-02         16.0        6.0   

       new_cases_per_million  population  
89000                  0.000  17500657.0  
89001                  0.343  17500657.0  
2021-07-10 00:00:00
True
      iso_code continent location       date  total_cases  new_cases  \
89465      SYR      Asia    Syria 2021-07-10      25766.0        0.0   

       new_cases_per_million  pop

       iso_code continent location        date  total_cases  new_cases  \
101000      ZMB    Africa   Zambia  2021-05-03      91722.0       29.0   
101001      ZMB    Africa   Zambia  2021-05-04      91804.0       82.0   

        new_cases_per_million  population  
101000                  1.577  18383956.0  
101001                  4.460  18383956.0  
2021-07-10 00:00:00
True
       iso_code continent  location       date  total_cases  new_cases  \
101068      ZMB    Africa    Zambia 2021-07-10     174789.0     2384.0   
101546      ZWE    Africa  Zimbabwe 2021-07-10      66853.0     1787.0   

        new_cases_per_million  population  assert_new_cases_per_million  
101068                129.678  18383956.0                    129.678291  
101546                120.232  14862927.0                    120.232038  
CPU times: user 1.77 s, sys: 401 ms, total: 2.18 s
Wall time: 4.91 s


/var/folders/6c/08v1kd5x5yjgw73zcdxqckzh0000gn/T/ipykernel_11209/4132050757.py:12: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior.  In a future version these will be considered non-comparable.Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.
  print(yesterday == chunk['date'].max())
/Users/lisa-mariekrause/Documents/01_Karriere/02_Bewerbungsunterlagen/Bewerbungen/Quantum_Black/Case_Prep/QB_case/lib/python3.9/site-packages/pandas/core/frame.py:3607: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._set_item(key, value)
/var/folders/6c/08v1kd5x5yjgw73zcdxqckzh0000gn/T/ipykernel_11209/4132050757.py:12: FutureWarning: Comparison of Timestamp with datetime.date is 

,iso_code,continent,location,date,total_cases,new_cases,new_cases_per_million,population,assert_new_cases_per_million
502,AFG,Asia,Afghanistan,2021-07-10,131586.0,0.0,0.000,3.892834e+07,0.000000
1016,OWID_AFR,NaN,Africa,2021-07-10,5914501.0,45374.0,33.846,1.340598e+09,33.846087
1518,ALB,Europe,Albania,2021-07-10,132587.0,7.0,2.432,2.877800e+06,2.432414
2020,DZA,Africa,Algeria,2021-07-10,145296.0,813.0,18.540,4.385104e+07,18.540038
2516,AND,Europe,Andorra,2021-07-10,14075.0,0.0,0.000,7.726500e+04,0.000000
...,...,...,...,...,...,...,...,...,...
99489,VNM,Asia,Vietnam,2021-07-10,28470.0,1862.0,19.129,9.733858e+07,19.129105
100131,OWID_WRL,NaN,World,2021-07-10,186459999.0,399937.0,51.308,7.794799e+09,51.308188
100588,YEM,Asia,Yemen,2021-07-10,6941.0,1.0,0.034,2.982597e+07,0.033528
101068,ZMB,Africa,Zambia,2021-07-10,174789.0,2384.0,129.678,1.838396e+07,129.678291


## 3. Descriptive Analysis

In [9]:
print(raw_TL.describe().T)
print(raw_TL.info())

                              count          mean           std    min  \
total_cases                   201.0  2.947312e+06  1.506004e+07    1.0   
new_cases                     201.0  6.099607e+03  3.196534e+04    0.0   
new_cases_per_million         200.0  7.921545e+01  1.609478e+02    0.0   
population                    203.0  1.171979e+08  6.567391e+08  809.0   
assert_new_cases_per_million  200.0  7.921546e+01  1.609478e+02    0.0   

                                    25%           50%           75%  \
total_cases                     13796.0  1.315860e+05  5.126850e+05   
new_cases                           0.0  9.100000e+01  1.295000e+03   
new_cases_per_million               0.0  1.928150e+01  7.534175e+01   
population                    2112816.0  9.537642e+06  3.291906e+07   
assert_new_cases_per_million        0.0  1.928178e+01  7.534184e+01   

                                       max  
total_cases                   1.864600e+08  
new_cases                     3.999370

## 4. Processing

In [10]:
#sorted(raw_TL[raw_TL['new_cases'].isna()], key=(lambda x: x['date']))
raw_TL['date'] = pd.to_datetime(raw_TL['date'])
raw_TL.info()
raw_TL[raw_TL['new_cases'].isna()].groupby('location')['date'].max()
set(raw_TL[raw_TL['continent'].isna()]['location'])

<class 'pandas.core.frame.DataFrame'>
Int64Index: 204 entries, 502 to 101546
Data columns (total 9 columns):
 #   Column                        Non-Null Count  Dtype         
---  ------                        --------------  -----         
 0   iso_code                      204 non-null    object        
 1   continent                     195 non-null    object        
 2   location                      204 non-null    object        
 3   date                          204 non-null    datetime64[ns]
 4   total_cases                   201 non-null    float64       
 5   new_cases                     201 non-null    float64       
 6   new_cases_per_million         200 non-null    float64       
 7   population                    203 non-null    float64       
 8   assert_new_cases_per_million  200 non-null    float64       
dtypes: datetime64[ns](1), float64(5), object(3)
memory usage: 15.9+ KB


{'Africa',
 'Asia',
 'Europe',
 'European Union',
 'International',
 'North America',
 'Oceania',
 'South America',
 'World'}

In [15]:
raw_TL_countries = raw_TL[raw_TL['continent'].isna() == False ]

In [33]:
result = raw_TL_countries.sort_values('new_cases_per_million', \
                                      ascending = False, \
                                     ignore_index = True)[0:9][['location', 'new_cases_per_million']]
result.set_index('location', inplace = True)

## 5. Output

In [34]:
result.to_json()

'{"new_cases_per_million":{"Cyprus":1081.075,"Mongolia":1073.12,"Tunisia":785.709,"Namibia":679.283,"Netherlands":601.055,"Cuba":595.941,"Fiji":564.452,"United Kingdom":468.948,"Kazakhstan":375.731}}'